<a href="https://colab.research.google.com/github/Clearbloo/Feynman_GNN/blob/main/dataset_builder_v2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Standard libraries
import os
import os.path as osp
import math
import numpy as np 
import time
import ast
import pandas as pd
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0

Mounted at /content/gdrive


# **Plan**


---


# Nodes:
Node list.

* 1 - Internal node
* 2 - Initial state nodes
* 3 - Final state nodes

The intention is to add information about the time direction. Could investigate the effect of including this.

e.g. 

```
node_features = [2, 2, 1, 1, 3, 3]
```



---


# Edge features/attributes:
Each edge comes with a list of features.

\begin{equation}
l = \begin{bmatrix}
m, & S, &LI^{W}_3, & LY, &RI^W_3, &RY,  &\text{colour}, & h, & \mathbf{p}
\end{bmatrix}
\end{equation}

Where $m$ is the on-shell mass. Examples of particles.

Lepton:

\begin{align}
  e^-_\uparrow &= \begin{bmatrix}
   m_e, & \frac{1}{2}, & -\frac{1}{2}, & -1,& 0,& -2, & False, & +1, & \mathbf{p}
  \end{bmatrix} \\[1em]
  e^-_\downarrow &= \begin{bmatrix}
   m_e, & \frac{1}{2}, & -\frac{1}{2}, & -1,& 0,& -2, & False, & -1, & \mathbf{p}
  \end{bmatrix}\\[1em]
  e^+_\uparrow &= \begin{bmatrix}
    m_e, & \frac{1}{2}, & 0, &+2, &+\frac{1}{2}, &+1, & False, & +1, & \mathbf{p}
  \end{bmatrix} \\[1em]
  e^+_\downarrow &= \begin{bmatrix}
    m_e, & \frac{1}{2}, & 0, &+2, &+\frac{1}{2}, &+1, & False, & -1, & \mathbf{p}
  \end{bmatrix}
\end{align}

Photon:

\begin{align}
  \gamma = \begin{bmatrix}
    0, &1, &0, &0, &0, &0, &0, &h, &\mathbf{p}
  \end{bmatrix}
\end{align}


---



# Edge Index (adjacency list):
A list of doublets that describe which edges connect to which.

e.g.
```
edge_index = [[1,2],[2,1],[1,3],[3,1]]
```




---

#**Define constants**

Always using natural units

In [2]:
m_e = 0.5110
m_mu = 105.6583755
alpha_QED = 1/137
q_e = np.sqrt(4*np.pi*alpha_QED)
num_edge_feat = 9

---
# **Creating Graph Representation Classes**





In [3]:
class adj_list:
    """
    Class for an directed graph
    """
    def __init__(self):
        self.graphsrc = []
        self.graphdest = []
 
    def add_edge(self, src, dest):
        self.graphsrc += [src]
        self.graphdest += [dest]
    
    def undirected(self):
      l = self.graphsrc
      self.graphsrc = l + self.graphdest
      self.graphdest = self.graphdest + l
 
    # Original function to print the graph
    def print_list(self):
        graph_list = [[self.graphsrc],[self.graphdest]]
        print(graph_list)

    def get_list(self):
      return [self.graphsrc,self.graphdest]


class Particle:
  """
  edge features vector
  l=[m,S,LIW3,LY,RIW3,RY,colour,h,p]
  masses in subsequent classes are given in MeV
  """
  def __init__(self, m: float, S: float, LIW: float, LY: float, RIW: float, RY:float, colour: int, h: int):
    self.feat = [m, S, LIW, LY, RIW, RY, colour, h]
    
  def get_feat(self):
    return self.feat

  def print_feat(self):
    print(self.feat)

In [4]:
#tests
graph = adj_list()
graph.add_edge(1,2)
graph.add_edge(3,1)
graph.undirected()
graph = graph.get_list()

print(graph)


[[1, 3, 2, 1], [2, 1, 1, 3]]


#Lepton classes:

In [5]:
class E_minus(Particle):
  """
  Class to construct the edge feautres of an electron
  l=[m,S,IW3,Y,colour,h,p]
  """
  def __init__(self, h):
    """
    h = helicity 
    p = 3-momentum vector
    """
    Particle.__init__(self, m=m_e, S=0.5, LIW=-0.5, LY=-1, RIW=0, RY=-2, colour=0, h=h)

class E_plus(Particle):
  """
  Class to construct the edge feautres of a left HELICITY positron
  """
  def __init__(self, h):
    """
    h = helicity 
    p = 3-momentum vector
    """
    Particle.__init__(self, m=m_e, S=0.5, LIW=0, LY=2, RIW=0.5, RY=1, colour=0, h=h)

class Mu_minus(Particle):
  """
  Class to construct the edge feautres of a left HELICITY muon
  l=[m,S,IW3,Y,colour,h,p]
  """
  def __init__(self, h):
    """
    h = helicity 
    p = 3-momentum vector
    """
    Particle.__init__(self, m=m_mu, S=0.5, LIW=-0.5, LY=-1, RIW=0, RY=-2, colour=0, h=h)

class Mu_plus(Particle):
  """
  Class to construct the edge feautres of a left HELICITY anti-muon
  """
  def __init__(self, h):
    """
    h = helicity 
    p = 3-momentum vector
    """
    Particle.__init__(self, m=m_mu, S=0.5, LIW=-0.5, LY=-1, RIW=0, RY=-2, colour=0, h=h)

#Boson classes:

In [6]:
class Photon(Particle):
  """
  edge features of a photon
  """
  def __init__(self, h):
    Particle.__init__(self, m=0, S=1, LIW=0, LY=0, RIW=0, RY=0, colour=0, h=h)

# **Create Tree-level QED Dataset**

We start with simple electron positron to muon antimuon QED tree level scattering. This removes the need for the t-channel diagram. And we only need to consider the s-channel diagram. 

Also only consider the matrix element $\mathcal{M}({\downarrow\uparrow\to\downarrow\uparrow})$. 

That is, left helicity electron meets right helicity positron to make a left helicity muon and right helicity antimuon.

The structure is to create a list for each of the features.

Then create a list of lists to represent the data for a singular graph

Then create a stacked list of lists of lists to represent the full dataset which then gets passed to pandas.dataframe

As a numpy array, this will be a 2D array (num_graphs, num_feature_type) with the objects as lists

In [7]:
#Setup: First make the dataframe a long list of arrays. 2 million data points
ang_res = 200
p_res = 10000
momenta_range=np.linspace(0, 10**5, p_res) 
dataframe = np.empty(shape=(ang_res*p_res,5),dtype=object)


#Node features
Feyn_vertex = [2,2,1,1,3,3]


#Adjacency lists
s_chan = adj_list()
s_chan.add_edge(0,2)
s_chan.add_edge(1,2)
s_chan.add_edge(2,3)
s_chan.add_edge(3,4)
s_chan.add_edge(3,5)

s_chan.undirected()
s_channel_edge_index = s_chan.get_list()

#Edge features (assuming all left helicity now for simplicity)
incoming_electron = E_minus(1).get_feat()
incoming_positron = E_plus(1).get_feat()
photon = Photon(1).get_feat()
outgoing_muon = Mu_minus(1).get_feat()
outgoing_antimuon = Mu_plus(1).get_feat()
edge_feat = [incoming_electron,
                 incoming_positron,
                 photon,
                 outgoing_muon,
                 outgoing_antimuon]
edge_feat += edge_feat

#Index to count the graph number
graph_count=0

for p in momenta_range:
  for theta in np.linspace(0,np.pi,ang_res):

    #Graph-level target
    Mfi_squared = (q_e**2*m_mu/(2*m_e)*(1-np.cos(theta)))**2

    #Create list of initial and final particle momenta with format [p,theta,phi]
    momenta = [[p,0,0],[p,np.pi,0],[p,theta,0],[p,np.pi+theta,0]]

    #Create the dataframe as an numpy array first
    dataframe[graph_count,0]=Feyn_vertex
    dataframe[graph_count,1]=s_channel_edge_index
    dataframe[graph_count,2]=edge_feat
    dataframe[graph_count,3]=Mfi_squared
    dataframe[graph_count,4]=momenta

    #increment the index
    graph_count += 1

dataframe = pd.DataFrame(dataframe, columns=['x','edge_index','edge_attr','y','p'], index=np.arange(0,dataframe.shape[0],1))

In [8]:
#save the file
filepath = '/content/gdrive/MyDrive/Part_III_Project/data/raw/'
os.makedirs(filepath, exist_ok=True)  
dataframe.to_csv(path_or_buf="/content/gdrive/MyDrive/Part_III_Project/data/raw/QED_data.csv")

In [14]:
print(dataframe['edge_attr'].tolist())

[[[0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1]], [[0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1]], [[0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [105.6583755, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, -0.5, -1, 0, -2, 0, 1], [0.511, 0.5, 0, 2, 0.5, 1, 0, 1], [0, 1, 0

Now for electron positron to electron positron, which requires the additional t-channel diagram.

In [ ]:
t_chan = adj_list()
t_chan.add_edge(1,3)
t_chan.add_edge(2,4)
t_chan.add_edge(3,4)
t_chan.add_edge(3,5)
t_chan.add_edge(4,6)

